In [82]:
import pandas as pd
import talib

In [83]:
daily = pd.read_excel('gbp_usd_1D.xlsx')
h1 = pd.read_excel('gbp_usd_1h.xlsx')

In [84]:
h1['atr'] = talib.ATR(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
    timeperiod=24
)

In [85]:
h1['rsi'] = talib.RSI(
    real=h1['close'],
    timeperiod=9
)

In [86]:
h1['ma'] = talib.MA(
    real=h1['close'],
    timeperiod=24
)

In [87]:
h1['true_range'] = talib.TRANGE(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
)

In [88]:
h1['range'] = h1['high'] - h1['low']

In [89]:
def candle_color(r):
    if r['open'] > r['close']:
        return 'r'
    if r['open'] < r['close']:
        return 'g'

    return 'y'


h1['candle_color'] = h1.apply(candle_color, axis=1)

In [90]:
def candle_type(r):
    ratio = r['true_range'] / r['atr']

    if ratio < 0.8:
        return 'spinning'
    if 1.2 > ratio >= 0.8:
        return 'standard'
    if 2.4 > ratio >= 1.2:
        return 'long'
    if ratio >= 2.4:
        return 'spike'


h1['candle_type'] = h1.apply(candle_type, axis=1)

In [91]:
h1 = h1.iloc[24:]

In [92]:
h1 = h1.drop(columns=['Unnamed: 0'])

In [93]:
h1 = h1.reset_index(drop=True)

In [94]:
def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'


h1['ma_status'] = h1.apply(set_ma_status, axis=1)

In [95]:
h1: pd.DataFrame = pd.DataFrame(h1)

In [96]:
h1['signal'] = pd.NA

In [97]:
ma_status = h1.at[0, 'ma_status']
for i, element in h1.iterrows():

    if ma_status == element['ma_status']:
        continue

    ma_status = element['ma_status']

    h1.loc[i, 'signal'] = 'buy' if ma_status == 'upper' else 'sell'



In [98]:
from dataclasses import dataclass,asdict


@dataclass
class TradeSignal:
    order_type: str
    entry_price: float
    sl: float
    tp0: float
    tp1: float
    tp2: float


signals = []
for i, element in h1.iterrows():
    
    if pd.isna(element['signal']):
        continue
    
    if element['signal'] == 'buy':
        signals.append(
            TradeSignal(
                order_type=element['signal'],
                entry_price=element['close'],
                sl=element['close'] - element['atr'],
                tp0=element['close'] + element['atr'] * 0.8,
                tp1=element['close'] + element['atr'] * 1.6,
                tp2=element['close'] + element['atr'] * 2.4,
            )
        )
        if element['signal'] == 'sell':
            signals.append(
                TradeSignal(
                    order_type=element['signal'],
                    entry_price=element['close'],
                    sl=element['close'] + element['atr'],
                    tp0=element['close'] - element['atr'] * 0.8,
                    tp1=element['close'] - element['atr'] * 1.6,
                    tp2=element['close'] - element['atr'] * 2.4,
                )
            )



In [99]:
h1['datetime'] = pd.to_datetime(h1['datetime'])

In [100]:
h1['signal'] = h1['signal'].fillna(0)

In [101]:
h1['signal'] = h1['signal'].apply(lambda s:-1 if s == 'sell' else 1)

In [102]:
def candle_type_id(candle_type):

    if candle_type == 'spinning':
        return 0
    if candle_type=='standard':
        return 1
    if candle_type == 'long':
        return 2
    if candle_type == 'spike':
        return 3

h1['candle_type_id'] = h1['candle_type'].apply(candle_type_id)

In [103]:
import backtrader as bt

In [219]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr', 'ma','signal','candle_type_id','rsi')  # add new lines
    params = (
        ('signal', -1),  # map DataFrame column "atr"
        ('atr', -1),  # map DataFrame column "atr"
        ('ma', -1),   # map DataFrame column "ma"
        ('candle_type_id', -1),   # map DataFrame column "ma"
        ('rsi', -1),   # map DataFrame column "ma"
    )
class SmaStrategy(bt.Strategy):


    def next(self):
        
        if self.data.signal[0] == 1:
            price = self.data.close[0]
            
            entry_price = price
            stop_loss   = entry_price - self.data.atr[0] * 2.4
            take_profit = entry_price + self.data.atr[0] * 4.8

            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size = 500
            )

In [220]:
data = CustomPandasData(dataname = h1.set_index("datetime"))

In [221]:

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(SmaStrategy)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
cerebro.broker.setcash(1000)
cerebro.broker.setcommission(commission=0.0001, leverage=500)
result =cerebro.run()


In [222]:
len(result[0].analyzers.trade.get_analysis())

8